In [1]:
import sys

sys.path.append("../")

import pandas as pd
from dotenv import load_dotenv

load_dotenv()
import os
import sqlite3

DB_PATH = os.getenv("DB_PATH")

conn = sqlite3.connect(DB_PATH)

In [4]:
only_country = False

# Individuals Regions
df_ind_regions = pd.read_sql_query("SELECT * FROM individuals_regions", conn)

# Years
df_ind = pd.read_sql_query("SELECT * FROM individuals_main_information", conn)
df_ind_year = df_ind[["individual_wikidata_id", "birthyear"]].drop_duplicates()
df_ind_year = df_ind_year.dropna()
df_ind_year = df_ind_year[df_ind_year['birthyear']<=1850]


temporal_resolution = 10

df_ind_year["productive_year"] = df_ind_year["birthyear"]  + 35
df_ind_year["decade"] = df_ind_year["productive_year"].apply(lambda x: round(x / temporal_resolution) * temporal_resolution)

df_catalogs_id = pd.read_sql_query("SELECT * FROM individual_identifiers", conn)
df_catalogs = pd.read_sql_query("SELECT * FROM identifiers", conn)

if only_country:
    df_catalogs = df_catalogs[['identifiers_wikidata_id', 'country_name']].dropna()
    df_cat = pd.merge(df_catalogs_id, df_catalogs, on = 'identifiers_wikidata_id')
    df_cat = df_cat[['individual_wikidata_id','individual_name', 'country_name']].drop_duplicates()
    df_cat = df_cat.groupby(['individual_name', 'individual_wikidata_id'])['country_name'].count().rename('score').reset_index()
    df_cat = df_cat.sort_values('score', ascending=False).reset_index(drop=True)
else:
    df_cat = df_catalogs_id.groupby(['individual_name', 'individual_wikidata_id'])['identifiers_wikidata_id'].count().rename('score').reset_index()
    df_cat = df_cat.sort_values('score', ascending=False).reset_index(drop=True)
    
df_final = pd.merge(df_ind_year, df_cat, on = 'individual_wikidata_id')
df_final = pd.merge(df_final, df_ind_regions, on = ['individual_wikidata_id', 'individual_name'])
df_final.to_csv('../data/df_individuals_score.csv')

In [6]:
df_final.sort_values('score', ascending=False).sample(3)

,individual_wikidata_id,birthyear,productive_year,decade,individual_name,score,region_code,region_name
280658,Q274614,1818.0,1853.0,1850,Élie-Abel Carrière,23,re_france,France
349085,Q14623869,1800.0,1835.0,1840,M. Abbeyson,1,re_western_europe,Western Europe
480191,Q64861214,1640.0,1675.0,1680,Gaspar Bouttats,39,re_low_countries,Low countries


In [7]:
df_final[(df_final['region_name']=='Italy')&(df_final['decade']<=500)].sort_values('score', ascending=False)

,individual_wikidata_id,birthyear,productive_year,decade,individual_name,score,region_code,region_name
191114,Q1541,-106.0,-71.0,-70,Cicero,205,re_italy,Italy
414730,Q1048,-99.0,-64.0,-60,Julius Caesar,200,re_italy,Italy
547123,Q1398,-69.0,-34.0,-30,Virgil,192,re_italy,Italy
35071,Q40939,-524.0,-489.0,-490,Aeschylus,188,re_italy,Italy
110641,Q2054,-4.0,31.0,30,Seneca,184,re_italy,Italy
...,...,...,...,...,...,...,...,...
380668,Q11690893,-150.0,-115.0,-120,Marcus Stallius,1,re_italy,Italy
142309,Q21798168,150.0,185.0,180,Gaios,1,re_italy,Italy
459953,Q1241543,50.0,85.0,80,Lucius Stertinius Avitus,1,re_italy,Italy
363325,Q16932930,330.0,365.0,360,Saint Sylvia of Aquitaine,1,re_italy,Italy
